# Requirements

If you have not created your Neo4J Sandbox yet, please run the pre-setup notebook to do sp.

# Setup

Install the necessary library in your Colab notebook environment and connect to your hosted Neo4J Sandbox.

In [13]:
!pip install neo4j

In [14]:
ip = "54.172.14.140"
bolt_port = "7687"
username = "neo4j"
password = "rifle-sponsor-beliefs"

In [15]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://" + ip + ":" + bolt_port, auth=(username, password))

print(driver.address) # your-sandbox-ip:your-sandbox-bolt-port

54.172.14.140:7687


# Citation Dataset Loading

In this notebook we're going to load the citation dataset into Neo4j.

## Reset database
Ensure you start with a clean and empty database in your Sandbox (it may be pre-loaded with toy data). To do so, run the following cell to delete all nodes and relationships.



In [ ]:
query = """
  CALL apoc.periodic.iterate(
    "MATCH (n) RETURN n", 
    "DETACH DELETE n", 
    {batchSize:1000}
  )
  yield batches, total return batches, total
"""

with driver.session() as session:
  result = session.run(query)
  for row in result:
    print(row)

<Record batches=3 total=2642>


## Create Constraints

First let's create some constraints to make sure we won't import duplicate data. We basically use an attribute as an ID for each type of node that we will create.

In [ ]:
with driver.session() as session:
    display(session.run("CREATE CONSTRAINT ON (a:Article) ASSERT a.index IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (a:Author) ASSERT a.name IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT ON (v:Venue) ASSERT v.name IS UNIQUE").consume().counters)

{'constraints_added': 1}

{'constraints_added': 1}

{'constraints_added': 1}

## Load the data

Now let's load the data into the database. We will create nodes and relationships for *Articles*, *Venues*, and *Authors* from JSON files that represent an extract of the DBLP dataset. 


In [ ]:
query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-0.json", "dblp-ref-1.json", "dblp-ref-2.json", "dblp-ref-3.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   RETURN value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 500, iterateList: true});
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=104 total=51956 timeTaken=19 committedOperations=51956 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 104, 'committed': 104, 'failed': 0, 'errors': {}} operations={'total': 51956, 'committed': 51956, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 43696, 'relationshipsCreated': 78585, 'nodesCreated': 43696, 'propertiesSet': 229418, 'relationshipsDeleted': 0, 'labelsRemoved': 0}>


Let's check that we did not create any *Article* without title.

In [ ]:
query = """
  call apoc.periodic.iterate(
    'MATCH (a:Article) WHERE not(exists(a.title)) RETURN a',
    'DETACH DELETE a',
    {batchSize:1000}
  )
  yield batches, total return batches, total
"""

with driver.session() as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=133 total=132357>


# Explore the data

Let's use the intuitive Neo4J Browser to explore the data that we have imported using Cypher queries. To access the Browser, go to your Sandbox control panel (at https://sandbox.neo4j.com/) and click the *Open* button.

Here are some queries left as exercise to explore your newly created DBLP database in the Neo4J Browser. Various solutions are valid for each question. Hint: use the "LIMIT" clause to avoid returning too many results and slowing down your queries.

- Show some *Articles* and their *Author*

In [ ]:
#@title Solution

MATCH (a:Article)-[:AUTHOR]-(b) 
RETURN a.title, b.name 
LIMIT 50

- Show some articles that talk about "random forests". Hint: use their *title* and *abstract* properties.

In [ ]:
#@title Solution

MATCH (a:Article) 
WHERE toLower(a.title) CONTAINS 'random forest' OR toLower(a.abstract) CONTAINS 'random forest'
RETURN a.title, a.abstract 
LIMIT 50

- Get the total number of *Article*, *Author* and *Venue* in the database. Hint: use nodes' "LABEL".

In [ ]:
#@title Solution

MATCH(n) 
RETURN LABELS(n), COUNT(n)

- Get the number of relationships for each type (*VENUE*, *CITED*, ...). Hint: use relationships' "TYPE".

In [ ]:
#@title Solution

MATCH ()-[r]->() 
RETURN TYPE(r), COUNT(r)

- Find the most cited *Article*.


In [ ]:
#@title Solution

MATCH (a:Article)<-[:CITED]-(b:Article) 
RETURN a.title, COUNT(b) as citations
ORDER BY citations desc
LIMIT 1

- Find the Top 10 *Author*s with the most collaborations. We count one collaboration as one article written by several authors.

In [ ]:
#@title Solution

MATCH (a:Author)<-[:AUTHOR]-(b:Article)-[:AUTHOR]->(c:Author) 
WHERE a <> c
RETURN a.name, COUNT(DISTINCT b) as collabs
ORDER BY collabs DESC
LIMIT 10

- Find the *Author* with whom "Salvatore Greco" has co-authored the most with.

In [ ]:
#@title Solution

MATCH (a:Author)<-[:AUTHOR]-(b:Article)-[:AUTHOR]->(c:Author) 
WHERE a.name = "Salvatore Greco"
RETURN a.name, c.name, COUNT(DISTINCT b) as collabs
ORDER BY collabs DESC
LIMIT 1

How could we use this last query as a basis to recommend future collaborations to Salvatore Greco? To make this easier, we will need to infer a graph of co-authorship.